In [2]:
import os
import cv2
import random
import numpy as np
from albumentations import HorizontalFlip, Rotate

In [ ]:
# 分割 T1 T2
archive_path = '../input/train/'
T2_path = '../input/T2Train/'
T1_path = '../input/T1Train/'

dirs = ['yes/', 'no/']
for dir in dirs:
  archive = os.listdir(archive_path + dir)
  T2 = os.listdir(T2_path + dir)
  for filename in archive:
    if filename not in T2:
      image = cv2.imread(archive_path + dir + filename, cv2.IMREAD_COLOR)
      cv2.imwrite(T1_path + dir + filename, image)

In [79]:
def seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed()

In [5]:
class CFG:
    n_fold = 4
    target_cols = ['label']

In [65]:
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

train = pd.read_csv('../input/RealTrain/' + 'data.csv')
folds = train.copy()
Fold = GroupKFold(n_splits=CFG.n_fold)
groups = folds['filename'].values
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[CFG.target_cols], groups)):
  folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
display(folds.groupby('fold').size())
display(folds)

fold
0    57
1    57
2    57
3    56
dtype: int64

,filename,label,T1,fold
0,Y1.jpg,1,1,2
1,Y10.jpg,1,1,1
2,Y101.jpg,1,1,3
3,Y103.jpg,1,1,1
4,Y104.jpg,1,1,0
...,...,...,...,...
222,No12.jpg,0,0,3
223,No14.jpg,0,0,1
224,No15.jpg,0,0,0
225,No21.jpg,0,0,0


In [80]:
all_path = '../input/RealTrain/train/'
t1_dst_path = '../input/T1TrainSameSize/'
t2_dst_path = '../input/T2TrainSameSize/'

new_folds = {'filename': [], 'label': [], 'fold': []}
for weight_type in ['T1', 'T2']:
    for fold in folds.groupby('fold'):
        fold_idx = fold[0]
        fold = fold[1]
        no_all = fold.label.value_counts()[0]
        yes_all = fold.label.value_counts()[1]
        T1_no = fold[(fold['label'] == 0) & (fold['T1'] == 1)].values
        T1_yes = fold[(fold['label'] == 1) & (fold['T1'] == 1)].values
        T2_no = fold[(fold['label'] == 0) & (fold['T1'] == 0)].values
        T2_yes = fold[(fold['label'] == 1) & (fold['T1'] == 0)].values
        T1_no_num = T1_no.shape[0]
        T1_yes_num = T1_yes.shape[0]
        T2_no_num = T2_no.shape[0]
        T2_yes_num = T2_yes.shape[0]
        print(f"T1_no: {T1_no_num} -> {no_all}")
        print(f"T1_yes: {T1_yes_num} -> {yes_all}")
        print(f"T2_no: {T2_no_num} -> {no_all}")
        print(f"T2_yes: {T2_yes_num} -> {yes_all}")

        for label in ['no', 'yes']:
            if weight_type == 'T1':
                dst_path = t1_dst_path
                if label == 'no':
                    num_all = no_all
                    num = T1_no_num
                    lst = T1_no
                else:
                    num_all = yes_all
                    num = T1_yes_num
                    lst = T1_yes
            else:
                dst_path = t2_dst_path
                if label == 'no':
                    num_all = no_all
                    num = T2_no_num
                    lst = T2_no
                else:
                    num_all = yes_all
                    num = T2_yes_num
                    lst = T2_yes
                    
            np.random.shuffle(lst)

            flag = False
            cnt = 0
            for aug_type in ['', 'h_', 'r10_', 'r20_', 'r30_']:
                if flag:
                    break
                
                for j in range(num):
                    if cnt == num_all:
                        flag = True
                        break
                    row = lst[j]
                    filename = row[0]
                    img = cv2.imread(all_path + filename, cv2.IMREAD_COLOR)
                    if aug_type == '':
                        image = img
                    elif aug_type == 'h_':
                        image = HorizontalFlip(p=1)(image=img)['image']
                    elif aug_type == 'r10_':
                        image = Rotate(p=1)(image=img, limit=(10, 11), border_mode=cv2.BORDER_CONSTANT, value=0)['image']
                    elif aug_type == 'r20_':
                        image = Rotate(p=1)(image=img, limit=(20, 21), border_mode=cv2.BORDER_CONSTANT, value=0)['image']
                    elif aug_type == 'r30_':
                        image = Rotate(p=1)(image=img, limit=(30, 31), border_mode=cv2.BORDER_CONSTANT, value=0)['image']
                    filename = aug_type + filename
                    cv2.imwrite(dst_path + 'train/' + filename, image)
                    new_folds['filename'].append(filename)
                    new_folds['label'].append(row[1])
                    new_folds['fold'].append(row[3])
                    cnt += 1

    new_folds = pd.DataFrame(new_folds)
    display(new_folds)
    new_folds.to_csv(dst_path + '/data.csv', index=False, sep=',')
    new_folds = {'filename': [], 'label': [], 'fold': []}
    

T1_no: 9 -> 22
T1_yes: 25 -> 35
T2_no: 13 -> 22
T2_yes: 10 -> 35
T1_no: 14 -> 20
T1_yes: 27 -> 37
T2_no: 6 -> 20
T2_yes: 10 -> 37
T1_no: 17 -> 23
T1_yes: 22 -> 34
T2_no: 6 -> 23
T2_yes: 12 -> 34
T1_no: 15 -> 21
T1_yes: 25 -> 35
T2_no: 6 -> 21
T2_yes: 10 -> 35


,filename,label,fold
0,no 95.jpg,0,0
1,18 no.jpg,0,0
2,N11.jpg,0,0
3,13 no.jpg,0,0
4,no.jpg,0,0
...,...,...,...
222,h_Y35.jpg,1,3
223,h_Y165.JPG,1,3
224,h_Y2.jpg,1,3
225,h_Y40.JPG,1,3


T1_no: 9 -> 22
T1_yes: 25 -> 35
T2_no: 13 -> 22
T2_yes: 10 -> 35
T1_no: 14 -> 20
T1_yes: 27 -> 37
T2_no: 6 -> 20
T2_yes: 10 -> 37
T1_no: 17 -> 23
T1_yes: 22 -> 34
T2_no: 6 -> 23
T2_yes: 12 -> 34
T1_no: 15 -> 21
T1_yes: 25 -> 35
T2_no: 6 -> 21
T2_yes: 10 -> 35


,filename,label,fold
0,no 6.jpg,0,0
1,21 no.jpg,0,0
2,47 no.jpg,0,0
3,3 no.jpg,0,0
4,no 7.jpeg,0,0
...,...,...,...
222,r20_Y251.JPG,1,3
223,r20_Y99.JPG,1,3
224,r20_Y65.JPG,1,3
225,r20_Y91.jpg,1,3


In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [ ]:
T1_PATH = '../input/T1Train/'
T2_PATH = '../input/T2Train/'

folds = {'filename': [], 'label': [], 'T1': []}
for filename in os.listdir(T1_PATH + 'yes/'):
  folds['filename'].append(filename)
  folds['label'].append(1)
  folds['T1'].append(1)
for filename in os.listdir(T1_PATH + 'no/'):
  folds['filename'].append(filename)
  folds['label'].append(0)
  folds['T1'].append(1)
for filename in os.listdir(T2_PATH + 'yes/'):
  folds['filename'].append(filename)
  folds['label'].append(1)
  folds['T1'].append(0)
for filename in os.listdir(T2_PATH + 'no/'):
  folds['filename'].append(filename)
  folds['label'].append(0)
  folds['T1'].append(0)

folds = pd.DataFrame(folds)
Fold = StratifiedKFold(n_splits=CFG.n_fold)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds['label'])):
  folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)

display(folds.groupby('fold').size())
display(folds)

folds.to_csv('../input/RealTrain/data.csv', index=False, sep=',')

In [59]:

# 打乱 T1
folds = pd.read_csv('../input/RealTrain/data.csv')
T1_idx = 2
T1 = folds['T1'].values

np.random.seed(42)
np.random.shuffle(T1)

folds['T1'] = T1

display(folds.groupby('fold').size())
display(folds)

folds.to_csv('../input/RealTrainRandom/data.csv', index=False, sep=',')

[1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 0 0 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1
 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 0
 1 1 0 1 1 0 1 1 1 1 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1
 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1
 1 1 1 0 1]


fold
0    58
1    57
2    56
3    56
dtype: int64

,filename,label,T1,fold
0,Y1.jpg,1,1,0
1,Y10.jpg,1,1,0
2,Y101.jpg,1,1,0
3,Y103.jpg,1,1,0
4,Y104.jpg,1,1,0
...,...,...,...,...
222,No12.jpg,0,1,3
223,No14.jpg,0,1,3
224,No15.jpg,0,1,3
225,No21.jpg,0,0,3


In [8]:
# 打乱 tumor
folds = pd.read_csv('../input/RealTrain/data.csv')
T1_idx = 2
tumor = folds['label'].values

np.random.seed(42)
np.random.shuffle(tumor)

folds['label'] = tumor

display(folds.groupby('fold').size())
display(folds)

folds.to_csv('../input/RealTrainRandomTumor/data.csv', index=False, sep=',')

fold
0    58
1    57
2    56
3    56
dtype: int64

,filename,label,T1,fold
0,Y1.jpg,1,1,0
1,Y10.jpg,0,1,0
2,Y101.jpg,1,1,0
3,Y103.jpg,0,1,0
4,Y104.jpg,0,1,0
...,...,...,...,...
222,No12.jpg,0,0,3
223,No14.jpg,1,0,3
224,No15.jpg,1,0,3
225,No21.jpg,1,0,3
